<a href="https://colab.research.google.com/github/walkerjian/dev/blob/main/AutoVibeAI_Prompted_Ada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!sudo apt-get update
!sudo apt-get install -y gnat


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 3s (134 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gnat is already the newest version (10.1ubuntu1).
0 upgraded, 0 newly installed

In [29]:
%%writefile sorting.ads
package Sorting is

   type List is array (1 .. 8) of Integer;  -- Defining the list type with a fixed size

   function Quick_Sort (Input : List) return List;    -- Quick sort function to sort the list

   procedure Verify_Safety (Input : List);    -- Function to verify safety standards

end Sorting;


Overwriting sorting.ads


In [26]:
%%writefile sorting.adb
package body Sorting is

   ---------------------------
   -- Implementation for Quick_Sort
   ---------------------------
   function Quick_Sort (Input : List) return List is

      function Sort (Arr : List; First, Last : Positive) return List is
         Temp_Arr : List := Arr;
         Pivot : Integer := Temp_Arr (Last);
         I : Positive := First;
         J : Positive := Last;
      begin
         while I < J loop
            while Temp_Arr(I) < Pivot and I < Last loop
               I := I + 1;
            end loop;
            while Temp_Arr(J) > Pivot and J > First loop
               J := J - 1;
            end loop;
            if I <= J then
               declare
                  Temp : Integer := Temp_Arr(I);
               begin
                  Temp_Arr(I) := Temp_Arr(J);
                  Temp_Arr(J) := Temp;
                  I := I + 1;
                  J := J - 1;
               end;
            end if;
         end loop;

         if First < J then
            Temp_Arr := Sort(Temp_Arr, First, J);
         end if;
         if I < Last then
            Temp_Arr := Sort(Temp_Arr, I, Last);
         end if;

         return Temp_Arr;
      end Sort;

   begin
      return Sort(Input, Input'First, Input'Last);
   end Quick_Sort;

   ---------------------------
   -- Implementation for Verify_Safety
   ---------------------------
   procedure Verify_Safety (Input : List) is
      Sorted_List : List := Quick_Sort (Input);
      Unsafe : Boolean := False;
   begin
      for I in Sorted_List'First + 1 .. Sorted_List'Last loop
         if Sorted_List (I) < Sorted_List (I - 1) then
            Unsafe := True;
            exit;
         end if;
      end loop;

      if Unsafe then
         raise Program_Error with "List is not sorted safely";
      end if;
   end Verify_Safety;

end Sorting;


Overwriting sorting.adb


In [27]:
!gnatmake sorting.adb

x86_64-linux-gnu-gcc-10 -c sorting.adb


In [28]:
%%writefile main.adb
with Ada.Text_IO;
with Sorting;

procedure Main is
   Input_List : Sorting.List := (5, 3, 8, 4, 2, 7, 6, 1);
   Sorted_List : Sorting.List;
begin
   Sorted_List := Sorting.Quick_Sort (Input_List);
   Ada.Text_IO.Put_Line ("Original List:");
   for I in Input_List'Range loop
      Ada.Text_IO.Put (Integer'Image (Input_List (I)) & " ");
   end loop;
   Ada.Text_IO.New_Line;

   Ada.Text_IO.Put_Line ("Sorted List:");
   for I in Sorted_List'Range loop
      Ada.Text_IO.Put (Integer'Image (Sorted_List (I)) & " ");
   end loop;
   Ada.Text_IO.New_Line;

   -- Verify Safety
   Sorting.Verify_Safety (Sorted_List);
   Ada.Text_IO.Put_Line ("List sorted and verified safely!");
exception
   when others =>
      Ada.Text_IO.Put_Line ("Error: List is not sorted safely.");
end Main;


Overwriting main.adb


In [30]:
!gnatmake sorting.adb main.adb
!./main

x86_64-linux-gnu-gcc-10 -c sorting.adb
x86_64-linux-gnu-gcc-10 -c main.adb
x86_64-linux-gnu-gnatbind-10 -x main.ali
x86_64-linux-gnu-gnatlink-10 main.ali
Original List:
 5  3  8  4  2  7  6  1 
Sorted List:
 1  2  3  4  5  6  7  8 
List sorted and verified safely!
